In [1]:
import pandas as pd
import numpy as np
import math
import operator
import pprint

In [2]:
df=pd.read_csv('weather.csv',index_col=0)
df

,Outlook,Temparature,Humidty,Wind,Play_Tennis
Day,,,,,
Day1,Sunny,Hot,High,Weak,NO
Day2,Sunny,Hot,High,Strong,NO
Day3,Overcast,Hot,High,Weak,YES
Day4,Rain,Mild,High,Weak,YES
Day5,Rain,Cool,Normal,Weak,YES
Day6,Rain,Cool,Normal,Strong,NO
Day7,Overcast,Cool,Normal,Strong,YES
Day8,Sunny,Mild,High,Weak,NO
Day9,Sunny,Cool,Normal,Weak,YES


In [ ]:
def entropy(p,n):
    positive=p/np.sum(p+n)
    negative=n/np.sum(p+n)
    e= -positive*math.log(positive,2)-negative*math.log(negative,2)
    return round(e,3)
def gain(df,entro):
    E=entropy(int(df[(df['Play_Tennis']=='YES')].count()['Outlook']),
          int(df[(df['Play_Tennis']=='NO')].count()['Outlook']))
    gain={}
    for col in df.columns[:-1]:
        temp=0
        i=0
        for val in df[col].unique():
            temp=temp- (int(df[df[col]==val].count()[col])/len(df))*colEntropy[col][i]
            i+=1
        gain[col]=round(E+temp,4)
    return gain

In [ ]:
gain(df,colEntropy)

In [ ]:

#max(gain(df,colEntropy).iteritem(), key=operator.itemgetter(1))[0]
inverse = [(value, key) for key, value in gain(df,colEntropy).items()]
print (inverse)

In [ ]:
E=entropy(int(df[(df['Play_Tennis']=='YES')].count()['Outlook']),
          int(df[(df['Play_Tennis']=='NO')].count()['Outlook']))
colEntropy={}
for col in df.columns[:-1]:
    temp=[]
    for val in df[col].unique():
        positive=int(df[(df[col]==val) & (df['Play_Tennis']=='YES')].count()[col])
        negative=int(df[(df[col]==val) & (df['Play_Tennis']=='NO')].count()[col])
        if(positive==0 or negative==0):
            temp.append(0)
        else:
            temp.append(entropy(int(positive),int(negative)))
    colEntropy[col]=temp

In [ ]:
colEntropy

In [ ]:
df[df['Outlook']=='Overcast'].count()['Outlook']

In [ ]:
df[(df['Play_Tennis']=='NO')].count()['Outlook']

In [ ]:
E=entropy(int(df[(df['Play_Tennis']=='YES')].count()['Outlook']),
          int(df[(df['Play_Tennis']=='NO')].count()['Outlook']))
E

In [ ]:
df.columns

In [ ]:
find_entropy_attribute(df,'Outlook')

In [ ]:
df[(df['Play_Tennis']=='YES')].count().iloc[0]

In [3]:
def Entropy(df):
    target=df.columns[-1]
    entropy=0
    for val in df[target].unique():
        count=df[(df[target]==val)].count().iloc[0]/len(df)
        entropy+=-count*math.log(count,2)
    return  round(entropy,4)
Entropy(df)        

0.9403

In [4]:
def Entropy_column(df,col):
    target=df.columns[-1]
    temp=[]
    for val in df[col].unique():
        entropy=0
        for label in df[target].unique():
            n=int(df[(df[col]==val) & (df[target]==label)].count()[col])
            total=len(df[(df[col]==val)])
            fraction=n/total
            if(fraction==0):
                entropy+=0
            else:
                entropy+=-fraction*math.log(fraction,2)
        temp.append(round(entropy,3))
    return temp

In [ ]:
def gain(df):
    E=Entropy(df)
    gain={}
    for col in df.columns[:-1]:
        temp=0
        i=0
        for val in df[col].unique():
            temp=temp- (int(df[df[col]==val].count()[col])/len(df))*colEntropy[col][i]
            i+=1
        gain[col]=round(E+temp,4)
    return gain
def rootnode(df):
    IG=gain(df)
    inverse = [(value, key) for key, value in IG.items()]
    return max(inverse)[1]

print(gain(df))
print(node(gain(df)))        

In [5]:
def treeNode(df):
    E=Entropy(df)
    IG = []
    for col in df.columns[:-1]:
        entropy=Entropy_column(df,col)
        IG.append(E-sum(entropy)/len(entropy))
    return df.keys()[:-1][np.argmax(IG)]

In [56]:
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)

In [57]:
subtable = get_subtable(df,'Outlook','Sunny')
subtable

,Outlook,Temparature,Humidty,Wind,Play_Tennis
0,Sunny,Hot,High,Weak,NO
1,Sunny,Hot,High,Strong,NO
2,Sunny,Mild,High,Weak,NO
3,Sunny,Cool,Normal,Weak,YES
4,Sunny,Mild,Normal,Strong,YES


In [26]:
np.unique(subtable['Play_Tennis'],return_counts=True)                        

(array(['NO', 'YES'], dtype=object), array([3, 2]))

In [60]:
def buildTree(df,tree=None): 
    target = list(df.columns[-1:])[0] #target-variable

    #get feature of max IG
    node = treeNode(df)

    #get unique values of node
    attValues = df[node].unique()
    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
    #loop to construct a tree by calling this function recursively. 
    for value in attValues:
        
        #subset of df after root node
        subtable = get_subtable(df,node,value)
        print(node,value)
        print(subtable)
        #get target variables with there count
        clValue,counts = np.unique(subtable[target],return_counts=True)                        
        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

In [61]:
tree=buildTree(df)
pprint.pprint(tree)

Outlook Sunny
  Outlook Temparature Humidty    Wind Play_Tennis
0   Sunny         Hot    High    Weak          NO
1   Sunny         Hot    High  Strong          NO
2   Sunny        Mild    High    Weak          NO
3   Sunny        Cool  Normal    Weak         YES
4   Sunny        Mild  Normal  Strong         YES
Humidty High
  Outlook Temparature Humidty    Wind Play_Tennis
0   Sunny         Hot    High    Weak          NO
1   Sunny         Hot    High  Strong          NO
2   Sunny        Mild    High    Weak          NO
Humidty Normal
  Outlook Temparature Humidty    Wind Play_Tennis
0   Sunny        Cool  Normal    Weak         YES
1   Sunny        Mild  Normal  Strong         YES
Outlook Overcast
    Outlook Temparature Humidty    Wind Play_Tennis
0  Overcast         Hot    High    Weak         YES
1  Overcast        Cool  Normal  Strong         YES
2  Overcast        Mild    High  Strong         YES
3  Overcast         Hot  Normal    Weak         YES
Outlook Rain
  Outlook Temparat

In [35]:
tree

{'Outlook': {'Sunny': {'Humidty': {'High': 'NO', 'Normal': 'YES'}},
  'Overcast': 'YES',
  'Rain': {'Wind': {'Weak': 'YES', 'Strong': 'NO'}}}}

In [ ]:
for col in df.columns:
    entro=Entropy_column(df,col)
    print(col,round(sum(entro)/len(entro),3))

In [36]:
def generate_edges(graph):
    edges = []
    for node in graph:
        for neighbour in graph[node]:
            edges.append((node, neighbour))

    return edges

print(generate_edges(tree))


[('Outlook', 'Sunny'), ('Outlook', 'Overcast'), ('Outlook', 'Rain')]


In [ ]:
eps = np.finfo(float).eps
from numpy import log2 as log
attribute = 'Outlook'
target_variables = df['Play_Tennis'].unique()  #This gives all 'Yes' and 'No'
variables = df[attribute].unique()    #This gives different features in that attribute (like 'Sweet')
entropy_attribute = 0
for variable in variables:
    entropy_each_feature = 0
    for target_variable in target_variables:
        num = len(df[attribute][df[attribute]==variable][df['Play_Tennis'] ==target_variable]) #numerator
        den = len(df[attribute][df[attribute]==variable])  #denominator
        fraction = num/(den+eps)  #pi
        entropy_each_feature += -fraction*log(fraction+eps) #This calculates entropy for one feature like 'Sweet'
    fraction2 = den/len(df)
    entropy_attribute += -fraction2*entropy_each_feature  

In [ ]:
entropy_attribute